In [2]:
from IPython.display import HTML
import numpy as np
import urllib3
from bs4 import BeautifulSoup as soup 
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

# Part I - Extraction of data from MTA's website
## Let's scraping the MTA website to get all the URLs that have weekly turnstile data in txt format

In [2]:
url = 'http://web.mta.info/developers/turnstile.html'
manager = urllib3.PoolManager()
response = manager.request('GET', url)

In [3]:
html = soup(response.data, 'html.parser')
html


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN"
   "http://www.w3.org/TR/html4/strict.dtd">

<html lang="en">
<head>
<title>mta.info | Turnstile Data</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<!--<meta http-equiv="X-UA-Compatible" content="IE=EmulateIE7">-->
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="/siteimages/favicon.ico" rel="shortcut icon"/>
<link href="/css/base.css" rel="stylesheet" type="text/css"/>
<link href="/css/grid.css" rel="stylesheet" type="text/css"/>
<link href="/css/topbar.css" rel="stylesheet" type="text/css"/>
<link href="/css/formalize.css" rel="stylesheet" type="text/css"/>
<!-- <link rel="stylesheet" type="text/css" href="/css/jquery.datepick.css"> -->
<!-- jQuery include should be at the top -->
<script language="javascript" src="/js/jquery-1.4.4.min.js" type="text/javascript"></script>
<link href="/css/template.css" rel="stylesheet" type="text/css"/>
<meta content="News" name="WT.cg_n">
<style media

In [4]:
html.head.title

<title>mta.info | Turnstile Data</title>

In [5]:
data_files = html.find_all('div', {'class':'span-84 last'})

In [6]:
len(data_files)

1

In [7]:
data_files[0]

<div class="span-84 last">
<h2>Data Files</h2>
<br/>
<a href="data/nyct/turnstile/turnstile_180310.txt">Saturday, March 10, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180303.txt">Saturday, March 03, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180224.txt">Saturday, February 24, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180217.txt">Saturday, February 17, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180210.txt">Saturday, February 10, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180203.txt">Saturday, February 03, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180127.txt">Saturday, January 27, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180120.txt">Saturday, January 20, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180113.txt">Saturday, January 13, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180106.txt">Saturday, January 06, 2018</a><br/><a href="data/nyct/turnstile/turnstile_171230.txt">Saturday, December 30, 2017</a><br/>

In [8]:
data_files[0].find_all('a')

[<a href="data/nyct/turnstile/turnstile_180310.txt">Saturday, March 10, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180303.txt">Saturday, March 03, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180224.txt">Saturday, February 24, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180217.txt">Saturday, February 17, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180210.txt">Saturday, February 10, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180203.txt">Saturday, February 03, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180127.txt">Saturday, January 27, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180120.txt">Saturday, January 20, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180113.txt">Saturday, January 13, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180106.txt">Saturday, January 06, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_171230.txt">Saturday, December 30, 2017</a>,
 <a href="data/nyct/turnstile/turnstile_171223.txt">Saturday, December 23, 

#### Store all the URLs in list

In [9]:
list_of_urls = []
for elem in data_files[0].find_all('a'):
    list_of_urls.append(elem.get('href'))

In [10]:
list_of_urls

['data/nyct/turnstile/turnstile_180310.txt',
 'data/nyct/turnstile/turnstile_180303.txt',
 'data/nyct/turnstile/turnstile_180224.txt',
 'data/nyct/turnstile/turnstile_180217.txt',
 'data/nyct/turnstile/turnstile_180210.txt',
 'data/nyct/turnstile/turnstile_180203.txt',
 'data/nyct/turnstile/turnstile_180127.txt',
 'data/nyct/turnstile/turnstile_180120.txt',
 'data/nyct/turnstile/turnstile_180113.txt',
 'data/nyct/turnstile/turnstile_180106.txt',
 'data/nyct/turnstile/turnstile_171230.txt',
 'data/nyct/turnstile/turnstile_171223.txt',
 'data/nyct/turnstile/turnstile_171216.txt',
 'data/nyct/turnstile/turnstile_171209.txt',
 'data/nyct/turnstile/turnstile_171202.txt',
 'data/nyct/turnstile/turnstile_171125.txt',
 'data/nyct/turnstile/turnstile_171118.txt',
 'data/nyct/turnstile/turnstile_171111.txt',
 'data/nyct/turnstile/turnstile_171104.txt',
 'data/nyct/turnstile/turnstile_171028.txt',
 'data/nyct/turnstile/turnstile_171021.txt',
 'data/nyct/turnstile/turnstile_171014.txt',
 'data/nyc

## Time to create a dataframe using above URLs
### Our analysis is only going to focus on year 2017. So we need to get the URLs for year 2017 and create a DataFrame

In [15]:
print(list_of_urls[0], "\nlength is", len(list_of_urls[0]))

data/nyct/turnstile/turnstile_180310.txt 
length is 40


In [20]:
list_of_urls[0][30:32]

'18'

#### So we need to grab the characters at index 30 & 31 to get the year

In [23]:
year_2017 = list()

In [26]:
for item in list_of_urls:
    if item[30:32] == '17':
        year_2017.append(item)


In [28]:
len(year_2017)

52

In [29]:
base_url = "http://web.mta.info/developers/"
base_url+year_2017[0]

'http://web.mta.info/developers/data/nyct/turnstile/turnstile_171230.txt'

In [32]:
mta = pd.DataFrame()
countdown = len(year_2017) + 1

for i in year_2017:
    temp = pd.read_csv(base_url+i)
    mta = mta.append(temp, ignore_index=True) #appending will result in index getting repeated and hence they need to be ignored
    countdown = countdown - 1
    print("Done with week",countdown)

Done with week 52
Done with week 51
Done with week 50
Done with week 49
Done with week 48
Done with week 47
Done with week 46
Done with week 45
Done with week 44
Done with week 43
Done with week 42
Done with week 41
Done with week 40
Done with week 39
Done with week 38
Done with week 37
Done with week 36
Done with week 35
Done with week 34
Done with week 33
Done with week 32
Done with week 31
Done with week 30
Done with week 29
Done with week 28
Done with week 27
Done with week 26
Done with week 25
Done with week 24
Done with week 23
Done with week 22
Done with week 21
Done with week 20
Done with week 19
Done with week 18
Done with week 17
Done with week 16
Done with week 15
Done with week 14
Done with week 13
Done with week 12
Done with week 11
Done with week 10
Done with week 9
Done with week 8
Done with week 7
Done with week 6
Done with week 5
Done with week 4
Done with week 3
Done with week 2
Done with week 1


In [33]:
mta.shape

(10258959, 11)

In [34]:
mta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10258959 entries, 0 to 10258958
Data columns (total 11 columns):
C/A                                                                     object
UNIT                                                                    object
SCP                                                                     object
STATION                                                                 object
LINENAME                                                                object
DIVISION                                                                object
DATE                                                                    object
TIME                                                                    object
DESC                                                                    object
ENTRIES                                                                 int64
EXITS                                                                   int64
dtypes: int64(2), 

We need to change DATE attribute's datatype to date time

In [42]:
mta["DATE"] = pd.to_datetime(mta["DATE"])

In [45]:
mta.dtypes

C/A                 object
UNIT                object
SCP                 object
STATION             object
LINENAME            object
DIVISION            object
DATE        datetime64[ns]
TIME                object
DESC                object
ENTRIES              int64
EXITS                int64
dtype: object

In [37]:
mta.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')

### So there are trailing spaces in EXITS column... We'll rename it.

In [38]:
mta.rename(columns={'EXITS                                                               ':'EXITS'}, inplace=True)

In [46]:
mta.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [47]:
len(mta["DATE"].unique())

364

#### The data is ready, however, it is best to save it so in case we need to shutdown the kernel or if it crashes, we won't need to re-run all the above script.
#### Pickling is the technique that is used to store python objects like lists, dictionaries, class objects, etc.
#### Pickling is the serializing and de-serializing of python objects to a byte stream. Unpicking is the opposite.

In [4]:
import pickle

In [52]:
pickle_out = open("mta_dataset","wb")

In [53]:
pickle.dump(mta, pickle_out)

In [5]:
#Open the pickle file and load it to the variable
pickle_in = open("mta_dataset","rb")
mta = pickle.load(pickle_in)


### So now we have the data for year 2017, we are ready for the analysis part.

***

# Part II - Analysis
## Understanding the data
#### Some details about the attributes

C/A      = Control Area (A002)

UNIT     = Remote Unit for a station (R051)

SCP      = Subunit Channel Position represents an specific address for a device (02-00-00)

STATION  = Represents the station name the device is located at

LINENAME = Represents all train lines that can be boarded at this station
                   Normally lines are represented by one character.  LINENAME 456NQR repersents train server for 4, 5, 6, N,Q, and R trains.

DIVISION = Represents the Line originally the station belonged to BMT, IRT, or IND   

DATE     = Represents the date (YYYY-MM-DD)

TIME     = Represents the time (hh:mm:ss) for a scheduled audit event

DESc     = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)
           1. Audits may occur more that 4 hours due to planning, or troubleshooting activities. 
           2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered. 

ENTRIES  = The comulative entry register value for a device

EXITS    = The cumulative exit register value for a device

**_The entries and exits aren’t counts per interval, but equivalent to an “odometer” reading for each device._**


** Which column represents a unique turnstile? **

At first I thought that the UNIT attribute is the turnstile, however, I found out that there are far fewer units at every station on an average just 2 or 3. Even a small station generally has at least 4 - 6 turnstiles and bigger ones have multiple entry/exit points so there could be 20 - 30 turnstiles.

After some Googling and going through some forums, I found that the SCP column represents the turnstiles. 


In [25]:
#number of unique units at every station
mta.groupby("STATION").UNIT.nunique()[:10]

STATION
1 AV               1
103 ST             3
103 ST-CORONA      1
104 ST             2
110 ST             1
111 ST             3
116 ST             3
116 ST-COLUMBIA    1
121 ST             1
125 ST             4
Name: UNIT, dtype: int64

In [26]:
mta.groupby("STATION").SCP.nunique().sort_values(ascending=False)[:10]

STATION
PATH NEW WTC       90
FULTON ST          80
34 ST-PENN STA     58
CHAMBERS ST        49
ATL AV-BARCLAY     48
TIMES SQ-42 ST     43
WALL ST            43
GRD CNTRL-42 ST    41
34 ST-HERALD SQ    41
59 ST COLUMBUS     41
Name: SCP, dtype: int64

In [28]:
mta.groupby("STATION").SCP.value_counts()[:50]

STATION        SCP     
1 AV           00-03-00    2204
               00-03-02    2204
               00-03-01    2203
               00-00-00    2202
               00-00-01    2202
               01-00-02    2196
               01-00-03    2196
               01-00-04    2195
               01-00-00    2194
               01-00-01    2193
103 ST         00-00-01    6596
               00-00-02    6596
               00-00-00    6579
               00-03-02    4409
               00-03-01    4404
               00-03-00    4394
103 ST-CORONA  00-00-00    2195
               00-05-00    2195
               00-05-01    2195
               00-06-00    2195
               00-06-01    2195
               00-00-01    2193
               00-00-02    2193
               00-00-03    2193
               00-00-04    2192
104 ST         00-00-01    4344
               00-00-02    4343
               00-00-00    4316
               00-06-01    2183
               00-06-00    2179
               0

**So clearly there are no unique SCP values for every station**

So we need to think of a combination to identify a unique turnstile.

In [29]:
#let's try with station as well as unit
mta.groupby(["STATION","UNIT"]).SCP.value_counts()[:50]


STATION        UNIT  SCP     
1 AV           R248  00-03-00    2204
                     00-03-02    2204
                     00-03-01    2203
                     00-00-00    2202
                     00-00-01    2202
                     01-00-02    2196
                     01-00-03    2196
                     01-00-04    2195
                     01-00-00    2194
                     01-00-01    2193
103 ST         R180  00-03-02    2215
                     00-00-01    2214
                     00-00-00    2213
                     00-00-02    2213
                     00-03-01    2209
                     00-03-00    2200
               R191  00-00-02    2195
                     00-03-01    2195
                     00-00-01    2194
                     00-03-00    2194
                     00-03-02    2194
                     00-00-00    2178
               R314  00-00-00    2188
                     00-00-01    2188
                     00-00-02    2188
103 ST-CORONA  R208 

We can see that SCPs are reapeted among different units.
**So STATION,UNIT,SCP should together identify a unique turnstile.**

**Also, let's check if there is any relation between STATION and C/A**

In [41]:
mta.groupby("STATION").["C/A"].nunique().value_counts()

SyntaxError: invalid syntax (<ipython-input-41-b6d691a02130>, line 1)

In [44]:
mta.rename(columns={'C/A':'CA'}, inplace=True)

In [45]:
mta.head()

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-23,03:00:00,REGULAR,6455840,2184987
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-23,07:00:00,REGULAR,6455856,2184995
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-23,11:00:00,REGULAR,6455899,2185082
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-23,15:00:00,REGULAR,6456038,2185156
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-23,19:00:00,REGULAR,6456327,2185197


In [72]:
mta.groupby("STATION").CA.nunique()

STATION
1 AV               2
103 ST             3
103 ST-CORONA      1
104 ST             2
110 ST             2
111 ST             4
116 ST             6
116 ST-COLUMBIA    1
121 ST             1
125 ST             6
135 ST             5
137 ST CITY COL    2
138/GRAND CONC     1
14 ST              7
14 ST-UNION SQ     5
145 ST             5
149/GRAND CONC     2
14TH STREET        1
15 ST-PROSPECT     2
155 ST             2
157 ST             2
161/YANKEE STAD    5
163 ST-AMSTERDM    1
167 ST             3
168 ST             3
169 ST             2
170 ST             3
174 ST             2
174-175 STS        2
175 ST             2
                  ..
SUTPHIN-ARCHER     1
SUTTER AV          1
SUTTER AV-RUTLD    1
THIRTY ST          1
THIRTY THIRD ST    1
TIMES SQ-42 ST     6
TOMPKINSVILLE      1
TREMONT AV         1
TWENTY THIRD ST    1
UNION ST           2
UTICA AV           2
V.CORTLANDT PK     1
VAN SICLEN AV      2
VAN SICLEN AVE     1
VERNON-JACKSON     3
W 4 ST-WASH SQ     2
W 8 S

So every station has at least 1 or more control areas.

We need to check that the control area and station has a one-to-one relationship.

In [79]:
mta.groupby("CA").STATION.nunique().sort_values(ascending=False).head(20)

CA
N408A    2
TRAM2    1
N103     1
N117     1
N116     1
N114     1
N113     1
N112A    1
N111     1
N110     1
N108     1
N102     1
N119     1
N101     1
N100     1
N098     1
N095A    1
N095     1
N094     1
N092     1
Name: STATION, dtype: int64

We can see that all but one CA correspond to one unique station.

So except for control area N408A, there is a one-to-one relationship between control area and station.

In [77]:
len(mta["CA"].unique())

737

In [76]:
mta.groupby("CA").STATION.nunique().value_counts()

1    736
2      1
Name: STATION, dtype: int64

This also proves that there is only one instance of 2 stations having the same control area.

<font color="red">**_So we now we can conclude that the a turnstile can be identified by the combination of (STATION, UNIT, SCP)_**</font>

Also, let's make sure that we remove this exception of CA N408A.

In [82]:
mta.drop(mta[mta["CA"] == "N408A"].index, inplace=True)

In [83]:
mta[mta["CA"] == "N408A"]

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS


We have successfully removed records with CA = N408A

***

## The next question is getting the actual number of entries and exits
I found out from a forum that the turnstile readings reset every now and then, however, the reading **do not completely reset to 0**.

I found this by plotting the numbers by date and time attributes in Tableau.

One startegy could be to get the difference in consecutive readings taken from a turnstile.

However, we first need to have the dataset sorted in the order of station.

## Which station has the most number of units?

In [54]:
mta.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-23,03:00:00,REGULAR,6455840,2184987
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-23,07:00:00,REGULAR,6455856,2184995
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-23,11:00:00,REGULAR,6455899,2185082
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-23,15:00:00,REGULAR,6456038,2185156
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-23,19:00:00,REGULAR,6456327,2185197


In [122]:
mta.groupby("STATION").UNIT.nunique().sort_values(ascending=False)[:5]

STATION
23 ST              6
CANAL ST           5
86 ST              5
GRD CNTRL-42 ST    4
125 ST             4
Name: UNIT, dtype: int64

## What is the total number of entries & exits across the subway system for August 1, 2017?

In [70]:
mta[mta["DATE"] == "2017-08-01"].ENTRIES.sum()

1176228207171

In [71]:
mta[mta["DATE"] == "2017-08-01"].EXITS.sum()

936411287545

## What station was the busiest on August 1, 2017? What turnstile was the busiest on that date?

In [57]:
mta['Busyness'] = mta["ENTRIES"] + mta["EXITS"]

In [66]:
mta[mta["DATE"]=="2017-08-01"].head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Busyness
4158549,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-08-01,00:00:00,REGULAR,6273793,2125421,8399214
4158550,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-08-01,04:00:00,REGULAR,6273806,2125422,8399228
4158551,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-08-01,08:00:00,REGULAR,6273848,2125522,8399370
4158552,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-08-01,12:00:00,REGULAR,6274008,2125779,8399787
4158553,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-08-01,16:00:00,REGULAR,6274328,2125860,8400188
4158554,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-08-01,20:00:00,REGULAR,6275094,2125925,8401019
4158591,A002,R051,02-00-01,59 ST,NQR456W,BMT,2017-08-01,00:00:00,REGULAR,5666136,1260041,6926177
4158592,A002,R051,02-00-01,59 ST,NQR456W,BMT,2017-08-01,04:00:00,REGULAR,5666143,1260044,6926187
4158593,A002,R051,02-00-01,59 ST,NQR456W,BMT,2017-08-01,08:00:00,REGULAR,5666165,1260105,6926270
4158594,A002,R051,02-00-01,59 ST,NQR456W,BMT,2017-08-01,12:00:00,REGULAR,5666302,1260244,6926546


In [74]:
mta[mta["DATE"] == "2017-08-01"].groupby("STATION").Busyness.sum().sort_values(ascending=False)[:1]

STATION
34 ST-HERALD SQ    87177496853
Name: Busyness, dtype: int64

So 34th St Herald Square station was the busiest one on 8/1/2017.

In [124]:
mta[(mta["DATE"] == "2017-08-01") & (mta["STATION"] == "34 ST-HERALD SQ")].groupby(["UNIT","SCP"]).Busyness.sum().sort_values(ascending=False)

UNIT  SCP     
R022  00-05-04    26440426447
R023  01-03-02    21630992658
      01-03-01    18318410339
R022  02-06-00    14343932923
      02-06-01      848927152
      01-06-00      676748738
R023  01-06-02      510463943
R022  02-00-07      470465782
R023  00-00-00      316149954
R022  00-05-03      293446317
      01-00-01      270737170
      01-00-00      266796889
R023  01-06-01      245009571
      01-06-00      201879740
R022  00-00-01      200989428
R023  00-03-05      145548108
R022  02-00-08      142139301
      01-00-02      133084936
R023  00-00-02      128733495
      00-00-03      119413948
R022  00-00-00      114843362
      00-05-05      111424569
      02-00-01      104054434
      01-06-01       92992556
      02-00-03       91577821
R023  01-00-01       90095306
      01-00-02       87941210
R022  00-05-00       85900259
      02-00-02       65283183
      00-03-04       56147299
      02-00-06       55295082
R023  01-00-00       53200172
      00-00-01       4956

In [88]:
mta.DESC.unique()

array(['REGULAR', 'RECOVR AUD'], dtype=object)

In [89]:
mta[mta["DESC"] == "RECOVR AUD"].head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Busyness
1715,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-29,11:00:00,RECOVR AUD,5820241,35214484,41034725
1716,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-29,15:00:00,RECOVR AUD,5820339,35215510,41035849
1757,A013,R081,01-03-00,49 ST,NQRW,BMT,2017-12-29,11:00:00,RECOVR AUD,4254479,4779304,9033783
1758,A013,R081,01-03-00,49 ST,NQRW,BMT,2017-12-29,15:00:00,RECOVR AUD,4254777,4779705,9034482
1799,A013,R081,01-03-01,49 ST,NQRW,BMT,2017-12-29,11:00:00,RECOVR AUD,5412317,2803052,8215369


In [96]:
test = mta.iloc[1710:1730]

In [97]:
test

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Busyness
1710,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-28,15:00:00,REGULAR,5819678,35210690,41030368
1711,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-28,19:00:00,REGULAR,5819940,35212395,41032335
1712,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-28,23:00:00,REGULAR,5820149,35213293,41033442
1713,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-29,03:00:00,REGULAR,5820202,35213435,41033637
1714,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-29,07:00:00,REGULAR,5820212,35213565,41033777
1715,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-29,11:00:00,RECOVR AUD,5820241,35214484,41034725
1716,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-29,15:00:00,RECOVR AUD,5820339,35215510,41035849
1717,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-29,19:00:00,REGULAR,5820599,35217084,41037683
1718,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-29,23:00:00,REGULAR,5820790,35218147,41038937
1719,A013,R081,01-03-00,49 ST,NQRW,BMT,2017-12-23,03:00:00,REGULAR,4246175,4770512,9016687


In [98]:
test["realEntries"] = test["ENTRIES"].diff()

/Users/akshayjam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [99]:
test

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Busyness,realEntries
1710,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-28,15:00:00,REGULAR,5819678,35210690,41030368,NaN
1711,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-28,19:00:00,REGULAR,5819940,35212395,41032335,262.0
1712,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-28,23:00:00,REGULAR,5820149,35213293,41033442,209.0
1713,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-29,03:00:00,REGULAR,5820202,35213435,41033637,53.0
1714,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-29,07:00:00,REGULAR,5820212,35213565,41033777,10.0
1715,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-29,11:00:00,RECOVR AUD,5820241,35214484,41034725,29.0
1716,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-29,15:00:00,RECOVR AUD,5820339,35215510,41035849,98.0
1717,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-29,19:00:00,REGULAR,5820599,35217084,41037683,260.0
1718,A013,R081,01-00-00,49 ST,NQRW,BMT,2017-12-29,23:00:00,REGULAR,5820790,35218147,41038937,191.0
1719,A013,R081,01-03-00,49 ST,NQRW,BMT,2017-12-23,03:00:00,REGULAR,4246175,4770512,9016687,-1574615.0


In [103]:
test = test.sort_values(["DATE", "TIME"])

In [104]:
test

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Busyness,realEntries
1719,A013,R081,01-03-00,49 ST,NQRW,BMT,2017-12-23,03:00:00,REGULAR,4246175,4770512,9016687,-1574615.0
1720,A013,R081,01-03-00,49 ST,NQRW,BMT,2017-12-23,07:00:00,REGULAR,4246193,4770540,9016733,18.0
1721,A013,R081,01-03-00,49 ST,NQRW,BMT,2017-12-23,11:00:00,REGULAR,4246241,4770640,9016881,48.0
1722,A013,R081,01-03-00,49 ST,NQRW,BMT,2017-12-23,15:00:00,REGULAR,4246362,4770960,9017322,121.0
1723,A013,R081,01-03-00,49 ST,NQRW,BMT,2017-12-23,19:00:00,REGULAR,4246671,4771318,9017989,309.0
1724,A013,R081,01-03-00,49 ST,NQRW,BMT,2017-12-23,23:00:00,REGULAR,4246934,4771614,9018548,263.0
1725,A013,R081,01-03-00,49 ST,NQRW,BMT,2017-12-24,03:00:00,REGULAR,4247112,4771665,9018777,178.0
1726,A013,R081,01-03-00,49 ST,NQRW,BMT,2017-12-24,07:00:00,REGULAR,4247124,4771676,9018800,12.0
1727,A013,R081,01-03-00,49 ST,NQRW,BMT,2017-12-24,11:00:00,REGULAR,4247170,4771788,9018958,46.0
1728,A013,R081,01-03-00,49 ST,NQRW,BMT,2017-12-24,15:00:00,REGULAR,4247349,4772078,9019427,179.0


In [105]:
test = test[["STATION","C/A","UNIT","SCP", "LINENAME","DIVISION","DATE","TIME","DESC","ENTRIES","EXITS","Busyness","realEntries"]]

In [106]:
test

,STATION,C/A,UNIT,SCP,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Busyness,realEntries
1719,49 ST,A013,R081,01-03-00,NQRW,BMT,2017-12-23,03:00:00,REGULAR,4246175,4770512,9016687,-1574615.0
1720,49 ST,A013,R081,01-03-00,NQRW,BMT,2017-12-23,07:00:00,REGULAR,4246193,4770540,9016733,18.0
1721,49 ST,A013,R081,01-03-00,NQRW,BMT,2017-12-23,11:00:00,REGULAR,4246241,4770640,9016881,48.0
1722,49 ST,A013,R081,01-03-00,NQRW,BMT,2017-12-23,15:00:00,REGULAR,4246362,4770960,9017322,121.0
1723,49 ST,A013,R081,01-03-00,NQRW,BMT,2017-12-23,19:00:00,REGULAR,4246671,4771318,9017989,309.0
1724,49 ST,A013,R081,01-03-00,NQRW,BMT,2017-12-23,23:00:00,REGULAR,4246934,4771614,9018548,263.0
1725,49 ST,A013,R081,01-03-00,NQRW,BMT,2017-12-24,03:00:00,REGULAR,4247112,4771665,9018777,178.0
1726,49 ST,A013,R081,01-03-00,NQRW,BMT,2017-12-24,07:00:00,REGULAR,4247124,4771676,9018800,12.0
1727,49 ST,A013,R081,01-03-00,NQRW,BMT,2017-12-24,11:00:00,REGULAR,4247170,4771788,9018958,46.0
1728,49 ST,A013,R081,01-03-00,NQRW,BMT,2017-12-24,15:00:00,REGULAR,4247349,4772078,9019427,179.0


Changing order of the columns...

In [107]:
mta = mta[["STATION","C/A","UNIT","SCP", "LINENAME","DIVISION","DATE","TIME","DESC","ENTRIES","EXITS","Busyness"]]

In [108]:
mta.head()

,STATION,C/A,UNIT,SCP,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Busyness
0,59 ST,A002,R051,02-00-00,NQR456W,BMT,2017-12-23,03:00:00,REGULAR,6455840,2184987,8640827
1,59 ST,A002,R051,02-00-00,NQR456W,BMT,2017-12-23,07:00:00,REGULAR,6455856,2184995,8640851
2,59 ST,A002,R051,02-00-00,NQR456W,BMT,2017-12-23,11:00:00,REGULAR,6455899,2185082,8640981
3,59 ST,A002,R051,02-00-00,NQR456W,BMT,2017-12-23,15:00:00,REGULAR,6456038,2185156,8641194
4,59 ST,A002,R051,02-00-00,NQR456W,BMT,2017-12-23,19:00:00,REGULAR,6456327,2185197,8641524


Sorting values...

In [113]:
mta = mta.sort_values(["STATION", "C/A", "UNIT", "SCP", "DATE", "TIME"])

In [125]:
mta.head()

,STATION,C/A,UNIT,SCP,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Busyness
10093563,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,03:00:00,REGULAR,11892866,13050144,24943010
10093564,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,07:00:00,REGULAR,11892926,13050308,24943234
10093565,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,11:00:00,REGULAR,11893253,13050807,24944060
10093566,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,15:00:00,REGULAR,11893870,13051562,24945432
10093567,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,19:00:00,REGULAR,11894473,13052475,24946948


In [127]:
mta2 = mta

In [131]:
mta2["Actual_Entries"]=mta2.groupby(["STATION", "C/A", "UNIT", "SCP"]).ENTRIES.diff()

In [132]:
mta2.head()

,STATION,C/A,UNIT,SCP,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Busyness,Actual_Entries
10093563,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,03:00:00,REGULAR,11892866,13050144,24943010,NaN
10093564,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,07:00:00,REGULAR,11892926,13050308,24943234,60.0
10093565,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,11:00:00,REGULAR,11893253,13050807,24944060,327.0
10093566,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,15:00:00,REGULAR,11893870,13051562,24945432,617.0
10093567,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,19:00:00,REGULAR,11894473,13052475,24946948,603.0


In [136]:
mta2[mta["STATION"].isin(["23 ST", "1 AV"])].to_csv("1AV_23ST.csv")

In [137]:
mta2["Actual_Exits"]=mta2.groupby(["STATION", "C/A", "UNIT", "SCP"]).EXITS.diff()

In [138]:
mta2["Busyness"] = 0

In [139]:
mta2.head()

,STATION,C/A,UNIT,SCP,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Busyness,Actual_Entries,Actual_Exits
10093563,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,03:00:00,REGULAR,11892866,13050144,0,NaN,NaN
10093564,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,07:00:00,REGULAR,11892926,13050308,0,60.0,164.0
10093565,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,11:00:00,REGULAR,11893253,13050807,0,327.0,499.0
10093566,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,15:00:00,REGULAR,11893870,13051562,0,617.0,755.0
10093567,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,19:00:00,REGULAR,11894473,13052475,0,603.0,913.0


In [140]:
mta2["Busyness"] = mta2["Actual_Entries"] + mta2["Actual_Exits"]

In [141]:
mta2.head()

,STATION,C/A,UNIT,SCP,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Busyness,Actual_Entries,Actual_Exits
10093563,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,03:00:00,REGULAR,11892866,13050144,NaN,NaN,NaN
10093564,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,07:00:00,REGULAR,11892926,13050308,224.0,60.0,164.0
10093565,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,11:00:00,REGULAR,11893253,13050807,826.0,327.0,499.0
10093566,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,15:00:00,REGULAR,11893870,13051562,1372.0,617.0,755.0
10093567,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,19:00:00,REGULAR,11894473,13052475,1516.0,603.0,913.0


In [143]:
mta2[["Actual_Entries", "Actual_Exits"]].fillna(0, inplace=True)

/Users/akshayjam/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [144]:
mta2["Busyness"] = 0

In [145]:
mta2["Busyness"] = mta2["Actual_Entries"] + mta2["Actual_Exits"]

In [1]:
mta2.head()

NameError: name 'mta2' is not defined